## Инкапсуляция


В программировании, под словом "инкапсуляция" могут понимать два различных понятия:

1. Механизм языка программирования по ограничению доступы к определеным компонентам объекта (иногда это называют **сокрытием**)
2. Возможность языка программирования "упаковывать" данные с методами, предназначенными для обработки этих данных

Давайте рассмотрим как в Python реализовано ограничение доступа к аттрибутам объекта.

In [6]:
import random
import string

symbols = list(string.printable)
random.shuffle(symbols)

class Person:
    def __init__(self, name, surname):
        self.name = name
        self.surname = surname
        k = random.randint(7, 15)
        self.password = ''.join(random.choices(symbols, k=k))

In [7]:
ivan = Person('Ivan', 'Ivanov')

In [8]:
ivan.password

'z):^{L\x0b'

Запрет на уровне соглашения

In [9]:
class Person:
    def __init__(self, name, surname):
        self.name = name
        self.surname = surname
        k = random.randint(7, 15)
        self._password = ''.join(random.choices(symbols, k=k))

In [10]:
ivan = Person('Ivan', 'Ivanov')

In [11]:
ivan._password

'p",1g";x'

In [12]:
ivan._password = 'Я поменялся!'

In [13]:
ivan._password

'Я поменялся!'

Аттрибут по прежнему доступен для просмотра и изменения:

In [14]:
ivan._password = 'Я поменял паспорт'
ivan._password # protected

'Я поменял паспорт'

Возможно ваша IDE даст вам понять о том что такие аттрибуты трогать нельзя:

![image](https://i.imgur.com/tbglQYt.png)

Чуть более строгий запрет

In [15]:
class Person:
    def __init__(self, name, surname):
        self.name = name # public
        self.surname = surname
        k = random.randint(7, 15)
        self.__password = ''.join(random.choices(symbols, k=k)) # private

In [16]:
ivan = Person('Ivan', 'Ivanov')

In [17]:
ivan.__password

AttributeError: 'Person' object has no attribute '__password'

Но это можно обойти:

In [18]:
ivan._Person__password

'eT %r[K\tZM'

In [19]:
ivan._Person__password = 15

In [20]:
ivan._Person__password

15

Однако

In [23]:
ivan.__password = 'Не понял...'

In [24]:
dir(ivan)

['_Person__password',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__password',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'name',
 'surname']

Т.е. происходит переименовывание нашего аттрибута. Важно что внутри класса он будет доступен по прежнему имени.

Поскольку метод - частный случай аттрибута, то все вышенаписаное относится и к методам. 

## Наследование. Super.

Под наследованием понимается возможность создать новые классы на основе уже существующих. Класс от которого наследуется текущий называют родительским.

Рассмотрим пример:

In [25]:
# Создадим класс на основе класса list, добавив два новых метода:
# 1. is_even - четна ли длина списка
# 2. square - возвращаем список все элементы которого квадрат данного

class MyList(list):

    def is_even(self):
        return len(self) % 2 == 0

    def square(self):
        return MyList([x ** 2 for x in self])


In [27]:
ls = MyList()
print(ls)

ls.extend([1,2,3,4,5])
print(ls)
print(ls.is_even())

ls.append(10)
print(ls)
print(ls.is_even())


print(ls.square())

[]
[1, 2, 3, 4, 5]
False
[1, 2, 3, 4, 5, 10]
True
[1, 4, 9, 16, 25, 100]


Таким образом, если аттрибут не найден в текущем классе, то Python идет искать в класс родитель, это касается и конструктора.

 - ```isinstance(x, A)```: является ли x экземпляром A (на 2-ом месте может стоять кортеж, тогда достаточно того чтобы x был экземпляром одного из элементов кортежа)
 - ```issubclass(A, B)```: является ли A потомком B (на 2-ом месте может стоять кортеж, тогда достаточно того чтобы A был потомком одного из элементов кортежа)
 - ```cls.mro()``` - пока что посмотреть на цепочку наследований (при множественном наследовании - список в порядоке поиска метода для данного объекта)

In [37]:
isinstance(ls, object)

True

In [38]:
issubclass(type(ls), object)

True

In [41]:
# Что происходит?

class A:
    pass

class B(A):
    pass

b = B()

In [43]:
B.mro()

[__main__.B, __main__.A, object]

Еще один пример

In [45]:
class Person:
    def __init__(self, name, surname):
        self.name = name
        self.surname = surname
        
class Student(Person):
    def __init__(self, name, surname, *args):
        Person.__init__(self, name, surname)
        self.courses = args

In [46]:
s = Student('Ivan', 'Sidorov', 'Math', 'Physics', 'Russian', 'History')
s.courses

('Math', 'Physics', 'Russian', 'History')

In [47]:
print(s.name, s.surname)

Ivan Sidorov


Функция **super** позволяет наследовать аттрибуты родителя.

In [48]:
class Person:
    def __init__(self, name, surname):
        self.name = name
        self.surname = surname
        
class Student(Person):
    def __init__(self, name, surname, *args):
        super(Student, self).__init__(name, surname)
        # Person.__init__(self, name, surname)
        self.courses = args

In [49]:
s = Student('Ivan', 'Sidorov', 'Math', 'Physics', 'Russian', 'History')
s.courses

('Math', 'Physics', 'Russian', 'History')

In [50]:
print(s.name, s.surname)

Ivan Sidorov


In [52]:
class Person:
    def __init__(self, name, surname):
        self.name = name
        self.surname = surname
        
class Student(Person):
    def __init__(self, name, surname, *args):
        super().__init__(name, surname)
        self.courses = args

In [53]:
s = Student('Ivan', 'Sidorov', 'Math', 'Physics', 'Russian', 'History')
s.courses

('Math', 'Physics', 'Russian', 'History')

In [54]:
print(s.name, s.surname)

Ivan Sidorov


Упражнение:

Создайте класс прямоугольника, который

1. Инициализирует стороны
2. Имеет метод для подсчета площади
3. Имеет метод для подсчета периметра

Затем, создайте класс квадрата, наследующийся от прямоугольника, таким образом чтобы все методы родителя
работали корректно для квадрата.

In [58]:
class Rectangle:
    def __init__(self, a, b):
        self.a = a
        self.b = b

    def area(self):
        print(f"My square: {self.a * self.b}")
        return self.a * self.b

    def perimeter(self):
        print(f"My perimeter: {2 * (self.a + self.b)}")
        return 2 * (self.a + self.b)

In [63]:
class Square(Rectangle):

    def __init__(self, a):
        self.a = a
        self.b = a

In [64]:
s = Square(10)
s

In [65]:
s.perimeter()

My perimeter: 40


40

In [66]:
s.area()

My square: 100


100

Важно: object - предок для всех классов, базовый класс

In [75]:
def f():
    pass

type(f).mro()

[type, object]

## Полиморфизм

Полиморфизм - единый интерфейс для сущностей разных типов.

Например, для оператора ```+```

In [76]:
print(1 + 2)
print('ab' + 'cd')

3
abcd


In [77]:
{1, 2, 3} & {3, 4, 5}

{3}

In [101]:
1 or 0 or 4

1

Для функций:

In [81]:
print(len([1,2,3]))
print(len({1,2,3}))

3
3


In [102]:
print(len({1: 'hi', 2:'bye'}))

2


Для классов:

In [103]:
class Cat:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def info(self):
        print(f"I am a cat. My name is {self.name}. I am {self.age} years old.")

    def make_sound(self):
        print("Meow")

class Dog:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def info(self):
        print(f"I am a dog. My name is {self.name}. I am {self.age} years old.")

    def make_sound(self):
        print("Bark")

In [104]:
cat1 = Cat("Kitty", 2.5)
dog1 = Dog("Fluffy", 4)

for animal in (cat1, dog1):
    animal.make_sound()
    animal.info()
    animal.make_sound()

Meow
I am a cat. My name is Kitty. I am 2.5 years old.
Meow
Bark
I am a dog. My name is Fluffy. I am 4 years old.
Bark


Для классов при наследовании:

In [106]:
class Animal:
    def __init__(self, name, legs, scariness):
        self.name = name 
        self.legs = legs
        self.scariness = scariness
    
    def introduce(self): 
        print("Hello! My name is %s!" % self.name)
    
    def sound(self):
        print("Sound!")

class Mammal(Animal):
    
    def __init__(self, name, scariness): 
        super().__init__(name=name, legs=4, scariness=scariness)

In [107]:
class Cat(Mammal):
    
    # переопределяем метод sound, чтобы кошка мяукала
    def sound(self): 
        print("Meow!")
    
class Dog(Mammal):
    
    # переопределяем метод sound, чтобы собака гавкала
    def sound(self): 
        print("Woof!")

class Cow(Mammal):
    
    # переопределяем метод sound, чтобы корова мычала
    def sound(self):
        print("Mooo!")

In [110]:
cat = Cat(name='Cat', scariness=2)
dog = Dog(name='Dog', scariness=3)
cow = Cow(name='Cow', scariness=1)

Одинаковый интерфейс (название функциии и аргументы), разные действия в зависимости от конкретного класса-потомка:


In [111]:
for animal in [cat, dog, cow]:
    print("%s wants to say something..." % animal.name)
    animal.sound()
    animal.introduce()

Cat wants to say something...
Meow!
Hello! My name is Cat!
Dog wants to say something...
Woof!
Hello! My name is Dog!
Cow wants to say something...
Mooo!
Hello! My name is Cow!


## Итераторы

Различают два понятия: **iterator** и **iterable**

**iterable** - объект по которому можно итерироваться. 

Примеры iterable:
- list
- tuple
- string
- dict
- set
- файл
- любой кастомный класс, в котором реализован метод ```__iter__```
- любой кастомный класс, в котором реализован метод ```__getitem__```

К объектам iterable можно применять циклы ```for``` и методы типа ```zip```, ```map``` в которых нужны последовательности элементов.

**iterator** - объект у которого определен метод ```__next__```.

In [112]:
ls = [1, 2, 3, 4, 5]
for x in ls:
    print(x, end=' ')
    # тело цикла

1 2 3 4 5 

Цикл for "под капотом"

In [113]:
ls = [1, 2, 3, 4, 5]
iterator = ls.__iter__()
while True:
    try:
        x = iterator.__next__()
    except StopIteration:
        break
    print(x, end=' ')
    # тело цикла

1 2 3 4 5 

В Python есть встроенные функции ```iter``` и ```next```, котоорые
эквивалентны ```__iter__``` и ```__next__```

In [115]:
ls = [1, 2, 3, 4, 5]
iterator = iter(ls)
while True:
    try:
        x = next(iterator)
    except StopIteration:
        break
    print(x, end=' ')
    # тело цикла

1 2 3 4 5 

В Python есть встроенные функции ```next``` и ```iter```, которые являются аналогами одноименных методов.

Давайте напишем собственный итератор, который ведет себя аналогично итератору для ```range```:

In [143]:
class MyRange:

   def __init__(self, start, end, step):
       self.start = start
       self.end = end
       self.step = step

   def __next__(self):
       if self.start < self.end:
           res = self.start
           self.start += self.step
           return res
       else:
           raise StopIteration

   def __iter__(self):
       return self


In [144]:
r = MyRange(1, 10, 2)

In [145]:
for x in r:
    print(x)

1
3
5
7
9


In [132]:
next(r)

StopIteration: 

То же самое, но итератор отдельно от объекта:

In [101]:
class MyIterator:
    def __init__(self, obj):
        self.start = obj.start
        self.end = obj.end
        self.step = obj.step
    def __next__(self):
        if self.start < self.end:
            ret = self.start
            self.start += self.step
            return ret
        else:
            raise StopIteration
        

class MyRange:
    def __init__(self, start, end, step=1):
        self.start = start
        self.end = end
        self.step = step
    def __iter__(self):
        return MyIterator(self)

In [146]:
r = MyRange(1, 10, 2)
for x in r:
    print(x, end=' ')

1 3 5 7 9 

In [ ]:
r.start

Теперь мы умеем прикрупчивать к своим классам возможность перечисления!